**Data Cleaning**

In [ ]:
import pandas as pd
import re

# Load dataset
df = pd.read_excel("/content/guvi_qa_table.xlsx")

# Preview
print(df.head())

# Example cleaning function
def clean_text(text):
    if pd.isna(text):
        return ""
    # Remove special characters (except basic punctuation)
    text = re.sub(r"[^a-zA-Z0-9\s.,!?]", "", text)
    # Convert to lowercase
    text = text.lower()
    # Remove extra spaces
    text = re.sub(r"\s+", " ", text).strip()
    return text

# Apply cleaning to all text columns
for col in df.select_dtypes(include=["object"]).columns:
    df[col] = df[col].apply(clean_text)

# Save cleaned version
df.to_excel("guvi_qa_table_cleaned.xlsx", index=False)


                                        Question  \
0                                  What is GUVI?   
1                 What is the main goal of GUVI?   
2            What domains do GUVI courses cover?   
3  What are the types of learning modes in GUVI?   
4                  What are GUVI's free courses?   

                                              Answer  
0  GUVI is an Indian ed-tech platform started in ...  
1  To democratize tech education for non-English-...  
2  Programming, Web Development, AI/ML, Cybersecu...  
3  Self-paced courses, Zen Class live programs, a...  
4  Python, Java, ChatGPT & AI, JavaScript for Beg...  


In [ ]:
# Apply cleaning to all text columns
for col in df.select_dtypes(include=["object"]).columns:
    df[col] = df[col].apply(clean_text)

# Remove duplicate rows
df = df.drop_duplicates()

# Save cleaned version
df.to_excel("guvi_qa_table_cleaned.xlsx", index=False)


** Tokenization **


In [ ]:
import pandas as pd
df = pd.read_excel("/content/guvi_qa_table_cleaned.xlsx")
print(df.columns)


Index(['Question', 'Answer'], dtype='object')


In [ ]:
from transformers import AutoTokenizer

# Load a multilingual tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

# Tokenize Question and Answer columns
df["Question_tokens"] = df["Question"].apply(lambda x: tokenizer.tokenize(x))
df["Answer_tokens"] = df["Answer"].apply(lambda x: tokenizer.tokenize(x))

# Token IDs (numerical format)
df["Question_token_ids"] = df["Question"].apply(lambda x: tokenizer.encode(x, truncation=True))
df["Answer_token_ids"] = df["Answer"].apply(lambda x: tokenizer.encode(x, truncation=True))

# Save tokenized dataset
df.to_excel("guvi_qa_table_tokenized.xlsx", index=False)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

**App launch**

In [ ]:
# Single Colab cell: GUVI multilingual chatbot (Excel-only, no generator)
# Install dependencies
!pip install -q streamlit transformers sentence-transformers langdetect pyngrok pandas openpyxl python-docx datasets accelerate

# ---------- Config: set your ngrok token here ----------
NGROK_AUTH_TOKEN = "313izp4YqqW3Zy6UZdS86QjZDd6_fkcwpNSy1mPVeYpcodaQ"

# ---------- App file content (app.py) ----------
app_code = r'''
import os
import streamlit as st
import pandas as pd
from langdetect import detect
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer, util
import torch
import numpy as np
from datetime import datetime

# ---------- SETTINGS ----------
EXCEL_PATH = "/content/guvi_qa_table_tokenized.xlsx"
NLLB_MODEL = "facebook/nllb-200-distilled-600M"
EMBED_MODEL = "paraphrase-multilingual-MiniLM-L12-v2"
SCORE_THRESHOLD = 0.28   # adjust as needed

SHORT_TO_NLLB = {
    "ta": "tam_Taml",
    "hi": "hin_Deva",
    "te": "tel_Telu",
    "ml": "mal_Mlym",
    "en": "eng_Latn"
}

# ---------- Load KB ----------
@st.cache_data
def load_kb(path=EXCEL_PATH):
    if not os.path.exists(path):
        raise FileNotFoundError(f"Excel KB not found at {path}. Upload the file or change EXCEL_PATH.")
    df = pd.read_excel(path)
    if "Question" not in df.columns or "Answer" not in df.columns:
        cols = [c.lower() for c in df.columns]
        qcol = None; acol = None
        for c in df.columns:
            if c.lower() in ("question","q","query","prompt"):
                qcol = c
            if c.lower() in ("answer","a","response","reply"):
                acol = c
        if qcol and acol:
            df = df[[qcol, acol]].rename(columns={qcol:"Question", acol:"Answer"})
        else:
            df = df.rename(columns={df.columns[0]:"Question"})
            df["Answer"] = df["Question"]
    df = df.dropna(subset=["Question","Answer"])
    return df.astype({"Question": str, "Answer": str}).reset_index(drop=True)

# ---------- Load models ----------
@st.cache_resource
def load_translation_model_and_tokenizer():
    tokenizer = AutoTokenizer.from_pretrained(NLLB_MODEL)
    model = AutoModelForSeq2SeqLM.from_pretrained(NLLB_MODEL)
    return tokenizer, model

@st.cache_resource
def load_embedder():
    return SentenceTransformer(EMBED_MODEL)

# ---------- NLLB translate helper ----------
def lang_to_nllb(code_or_short):
    return SHORT_TO_NLLB.get(code_or_short, code_or_short)

def translate_nllb(text, tgt_nllb, src_nllb=None, tokenizer=None, model=None):
    if tokenizer is None or model is None:
        tokenizer, model = load_translation_model_and_tokenizer()
    bos_id = None
    try:
        bos_id = tokenizer.lang_code_to_id.get(tgt_nllb, None)
    except Exception:
        bos_id = None
    if bos_id is None:
        token_string_candidates = [f"<{tgt_nllb}>", tgt_nllb]
        for tokstr in token_string_candidates:
            tok_id = tokenizer.convert_tokens_to_ids(tokstr)
            if tok_id != tokenizer.unk_token_id:
                bos_id = tok_id
                break
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    gen_kwargs = {"max_length": 512}
    if bos_id is not None and bos_id != tokenizer.unk_token_id:
        gen_kwargs["forced_bos_token_id"] = bos_id
    translated = model.generate(**inputs, **gen_kwargs)
    return tokenizer.batch_decode(translated, skip_special_tokens=True)[0]

# ---------- Embedding prep & search ----------
@st.cache_resource
def prepare_embeddings(kb_questions):
    embedder = load_embedder()
    return embedder.encode(kb_questions, convert_to_tensor=True)

def semantic_search(query_en, kb_questions, kb_embs, top_k=3):
    embedder = load_embedder()
    q_emb = embedder.encode(query_en, convert_to_tensor=True)
    scores = util.pytorch_cos_sim(q_emb, kb_embs)[0].cpu().numpy()
    idxs = np.argsort(-scores)[:top_k]
    return [(int(i), float(scores[i]), kb_questions[int(i)]) for i in idxs]

# ---------- Streamlit UI ----------
st.set_page_config(page_title="GUVI Multilingual Chatbot (Excel-only)", layout="wide")
st.title("🤖 GUVI Multilingual Chatbot — Excel KB Only")

# Load KB
try:
    df = load_kb()
except FileNotFoundError as e:
    st.error(str(e))
    st.stop()

tokenizer_nllb, nllb_model = load_translation_model_and_tokenizer()
questions = df["Question"].tolist()
answers = df["Answer"].tolist()
kb_embs = prepare_embeddings(questions)

st.sidebar.header("KB Info")
st.sidebar.write(f"Loaded {len(questions)} Q/A rows.")
st.sidebar.write(f"Embedding model: {EMBED_MODEL}")
st.sidebar.write(f"Translation model: {NLLB_MODEL}")
if st.sidebar.checkbox("Show sample rows"):
    st.sidebar.write(df.head())

user_input = st.text_input("Ask (Tamil/Hindi/Telugu/Malayalam/English):", "")

if st.button("Ask") and user_input.strip():
    # Detect language
    try:
        detected_short = detect(user_input)
    except Exception:
        detected_short = "en"
    src_nllb = lang_to_nllb(detected_short)
    tgt_nllb = "eng_Latn"

    # Translate query to English
    if detected_short != "en":
        try:
            query_en = translate_nllb(user_input, tgt_nllb, src_nllb, tokenizer=tokenizer_nllb, model=nllb_model)
        except Exception:
            query_en = user_input
    else:
        query_en = user_input

    # Semantic search
    matches = semantic_search(query_en, questions, kb_embs, top_k=3)
    best_idx, best_score, _ = matches[0]

    if best_score >= SCORE_THRESHOLD:
        final_answer_en = answers[best_idx]
    else:
        final_answer_en = "not upto my things"

    # Translate back
    if detected_short != "en":
        try:
            answer_local = translate_nllb(final_answer_en, lang_to_nllb(detected_short), src_nllb="eng_Latn", tokenizer=tokenizer_nllb, model=nllb_model)
        except Exception:
            answer_local = final_answer_en
    else:
        answer_local = final_answer_en

    st.markdown("**Answer:**")
    st.write(answer_local)

    # Debug matches
    if st.checkbox("Show top matches (debug)"):
        for i, s, txt in matches:
            st.write(f"- (score={s:.3f}) Q: {questions[i]}")

    # Log query
    log_row = {
        "timestamp": datetime.utcnow().isoformat(),
        "user_query": user_input,
        "query_en": query_en,
        "detected_lang": detected_short,
        "best_score": best_score,
        "answer_en": final_answer_en,
        "answer_local": answer_local
    }
    log_path = "/content/guvi_chat_logs.csv"
    pd.DataFrame([log_row]).to_csv(log_path, mode="a", header=not os.path.exists(log_path), index=False)
    st.sidebar.write(f"Logged query to {log_path}")
'''

# Write app.py to disk
with open("app.py", "w", encoding="utf-8") as f:
    f.write(app_code)

# ---------- Create requirements.txt ----------
requirements = """streamlit
transformers
sentence-transformers
langdetect
pyngrok
pandas
openpyxl
python-docx
datasets
accelerate
"""
with open("requirements.txt", "w") as f:
    f.write(requirements)


In [ ]:
# Install dependencies
!pip install -q streamlit transformers sentence-transformers langdetect pyngrok pandas openpyxl python-docx datasets accelerate


In [ ]:
!ngrok config add-authtoken 313izp4YqqW3Zy6UZdS86QjZDd6_fkcwpNSy1mPVeYpcodaQ


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!pkill -f ngrok



In [ ]:
from pyngrok import ngrok
import time
import subprocess

# Kill any old tunnels and processes
ngrok.kill()

# Start Streamlit app in background
process = subprocess.Popen(["streamlit", "run", "app.py", "--server.port", "8501"])

# Wait a bit for Streamlit to start
time.sleep(5)

# Start ngrok tunnel
public_url = ngrok.connect(8501)
print("✅ Public Streamlit URL:", public_url)


✅ Public Streamlit URL: NgrokTunnel: "https://544a623d2f93.ngrok-free.app" -> "http://localhost:8501"
